<a href="https://colab.research.google.com/github/Bimal-Kumar-002/Topic-Modling-using-Machine-Learning/blob/master/CRF_for_Name_Entity_Recogniton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-crfsuite

     |████████████████████████████████| 747kB 3.0MB/s 


# Import all important library 

In [ ]:
import pandas as pd # for data analysis
import numpy as np # for mathematical calculation
import pycrfsuite # for topic modeling Condational random fields
from bs4 import BeautifulSoup as bs # for finding the imporatnat data from xml file
from bs4.element import Tag
import codecs #
import nltk

In [ ]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

## # Read data file and parse the XML

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/My\ Drive/

/content/gdrive/My Drive


In [ ]:
%cd ./NLP/Dataset/Name_Entity_Recognition_Data & text data/

/content/gdrive/My Drive/NLP/Dataset/Name_Entity_Recognition_Data & text data


In [ ]:

with codecs.open("reuters.xml", "r", "utf-8") as infile:
    soup = bs(infile, "html5lib")

docs = []
for elem in soup.find_all("document"):
    texts = []

    # Loop through each child of the element under "textwithnamedentities"
    for c in elem.find("textwithnamedentities").children:
        if type(c) == Tag:
            if c.name == "namedentityintext":
                label = "N"  
            else:
                label = "I"  
            for w in c.text.split(" "):
                if len(w) > 0:
                    texts.append((w, label))
    docs.append(texts)

    Named entity->N & irrelevant word->I


In [ ]:
docs[0:2][:2]

[[('Paxar', 'N'),
  ('Corp', 'N'),
  ('said', 'I'),
  ('it', 'I'),
  ('has', 'I'),
  ('acquired', 'I'),
  ('Thermo-Print', 'N'),
  ('GmbH', 'N'),
  ('of', 'I'),
  ('Lohn', 'N'),
  (',', 'I'),
  ('West', 'N'),
  ('Germany', 'N'),
  (',', 'I'),
  ('a', 'I'),
  ('distributor', 'I'),
  ('of', 'I'),
  ('Paxar', 'N'),
  ('products,', 'I'),
  ('for', 'I'),
  ('undisclosed', 'I'),
  ('terms.', 'I')],
 [('Key', 'N'),
  ('Tronic', 'N'),
  ('corp', 'N'),
  ('said', 'I'),
  ('it', 'I'),
  ('has', 'I'),
  ('received', 'I'),
  ('contracts', 'I'),
  ('to', 'I'),
  ('provide', 'I'),
  ('seven', 'I'),
  ('original', 'I'),
  ('equipment', 'I'),
  ('manufacturers', 'I'),
  ('with', 'I'),
  ('which', 'I'),
  ('it', 'I'),
  ('has', 'I'),
  ('not', 'I'),
  ('done', 'I'),
  ('business', 'I'),
  ('recently', 'I'),
  ('with', 'I'),
  ('over', 'I'),
  ('300,000', 'I'),
  ('computer', 'I'),
  ('keyboards', 'I'),
  ('for', 'I'),
  ('delivery', 'I'),
  ('within', 'I'),
  ('the', 'I'),
  ('next', 'I'),
  ('12', 'I'

[In corpus linguistics, part-of-speech tagging (POS tagging or PoS tagging or POST), also called grammatical tagging is the process of marking up a word in a text (corpus) as corresponding to a particular part of speech,  based on both its definition and its context. A simplified form of this is commonly taught to school-age children, in the identification of words as nouns, verbs, adjectives, adverbs, etc](https://en.wikipedia.org/wiki/Part-of-speech_tagging)

In [ ]:
def tag_PoS_in_documnets(docs):
  data = []
  for i , doc in enumerate(docs):
    tokens = [t for t,label in doc]
    # Pos tagging
    tagged = nltk.pos_tag(tokens)
    data.append([(w,pos,label) for (w,label),(word,pos) in zip(doc,tagged)])
  return data # orderd (word,pos, label)

In [ ]:
data = tag_PoS_in_documnets(docs)

In [ ]:
data[0:2][:1]

[[('Paxar', 'NNP', 'N'),
  ('Corp', 'NNP', 'N'),
  ('said', 'VBD', 'I'),
  ('it', 'PRP', 'I'),
  ('has', 'VBZ', 'I'),
  ('acquired', 'VBN', 'I'),
  ('Thermo-Print', 'NNP', 'N'),
  ('GmbH', 'NNP', 'N'),
  ('of', 'IN', 'I'),
  ('Lohn', 'NNP', 'N'),
  (',', ',', 'I'),
  ('West', 'NNP', 'N'),
  ('Germany', 'NNP', 'N'),
  (',', ',', 'I'),
  ('a', 'DT', 'I'),
  ('distributor', 'NN', 'I'),
  ('of', 'IN', 'I'),
  ('Paxar', 'NNP', 'N'),
  ('products,', 'NN', 'I'),
  ('for', 'IN', 'I'),
  ('undisclosed', 'JJ', 'I'),
  ('terms.', 'NN', 'I')]]

## Generating Features
We can now continue to generate more features for each of the tokens in the dataset.Some commonly used features for a word 'w' in named entity recognition -
 
*    The word w itself (converted to lowercase for normalisation)
*    The prefix/suffix of w (e.g. -ion)
*    The words surrounding w, such as the previous and the next word
*    Whether w is in uppercase or lowercase 
*    Whether w is a number, or contains digits 
*    The POS tag of w, and those of the surrounding words 
*    Whether w is or contains a special character (e.g. hypen, dollar sign)

[Reference of code](https://github.com/scrapinghub/python-crfsuite/blob/master/examples/CoNLL%202002.ipynb)


In [ ]:
def word2features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]

    # Common features for all words
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag
    ]

    # Features for words that are not
    # at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:word.isdigit=%s' % word1.isdigit(),
            '-1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not
    # at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:word.isdigit=%s' % word1.isdigit(),
            '+1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features

### Data prepration

In [ ]:
# extracting features in documents
def extract_features(doc):
    return [word2features(doc, i) for i in range(len(doc))]

# generating the list of labels for each document
def get_labels(doc):
    return [label for (token, postag, label) in doc]

X = [extract_features(doc) for doc in data]
y = [get_labels(doc) for doc in data]


In [ ]:
print("feature X & label y : \n",format((X[1][0] , y[1][0])))

feature X & label y : 
 (['bias', 'word.lower=key', 'word[-3:]=Key', 'word[-2:]=ey', 'word.isupper=False', 'word.istitle=True', 'word.isdigit=False', 'postag=NNP', 'BOS', '+1:word.lower=tronic', '+1:word.istitle=True', '+1:word.isupper=False', '+1:word.isdigit=False', '+1:postag=NNP'], 'N')


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

### CRF model
[CRF Documentation](http://www.chokkan.org/software/crfsuite/manual.html#idp8849114176)

In [ ]:
trainer = pycrfsuite.Trainer(verbose=True)

# Submit training data to the trainer
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

# Set the parameters of the model
trainer.set_params({
    # coefficient for L1 penalty
    'c1': 0.2,
    # coefficient for L2 penalty
    'c2': 0.02,  
    'max_iterations': 400,
    'feature.possible_transitions': True  # whether to include transitions that are possible, but not observed
})
# Save the model
trainer.train('crf.model')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 15163
Seconds required: 0.039

L-BFGS optimization
c1: 0.200000
c2: 0.020000
num_memories: 6
max_iterations: 400
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 6413.105838
Feature norm: 1.000000
Error norm: 7005.598321
Active features: 14693
Line search trials: 1
Line search step: 0.000035
Seconds required for this iteration: 0.014

***** Iteration #2 *****
Loss: 5219.878817
Feature norm: 0.851398
Error norm: 6307.519631
Active features: 14669
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.008

***** Iteration #3 *****
Loss: 4708.542467
Feature norm: 0.816564
Error norm: 14633.725218
Active features: 6549
Line search trials: 2
Line search step: 0.500000
Seconds required for this 

In [ ]:
tagger = pycrfsuite.Tagger()
tagger.open('crf.model')
y_pred = [tagger.tag(xseq) for xseq in X_test]

# random sample in the testing set
i = 6
for x, y in zip(y_pred[i], [x[1].split("=")[1] for x in X_test[i]]):
    print("%s (%s)" % (y, x))

britain (N)
intends (I)
to (I)
negotiate (I)
further (I)
agreements (I)
on (I)
international (I)
securities (I)
regulation (I)
to (I)
match (I)
those (I)
now (I)
in (I)
force (I)
with (I)
the (I)
u.s (N)
. (I)
and (I)
japan (N)
, (I)
government (I)
sources (I)
said. (I)
the (I)
department (I)
of (I)
trade (N)
and (N)
industry (N)
said (I)
earlier (I)
it (I)
had (I)
concluded (I)
a (I)
memorandum (I)
of (I)
understanding (I)
with (I)
japans (N)
finance (N)
ministry (N)
to (I)
swap (I)
supervisory (I)
and (I)
investigatory (I)
information. (I)
the (I)
agreement (I)
covers (I)
stocks, (I)
shares (I)
and (I)
government (I)
bonds, (I)
but (I)
not (I)
commodity (I)
futures, (I)
whose (I)
regulation (I)
in (I)
japan (N)
does (I)
not (I)
come (I)
under (I)
the (I)
finance (N)
ministry (N)
. (I)
it (I)
therefore (I)
does (I)
not (I)
cover (I)
some (I)
transactions (I)
on (I)
the (I)
london (N)
international (N)
financial (N)
futures (N)
exchange (N)
and (I)
the (I)
london (N)
commodities (I)
an

## Accuracy of the model

In [ ]:
from sklearn.metrics import classification_report, f1_score

In [ ]:
# mapping the labels 
labels = {"N": 1, "I": 0}

# Convert the sequences of tags into a 1-dimensional array
predictions = np.array([labels[tag] for row in y_pred for tag in row])
truths = np.array([labels[tag] for row in y_test for tag in row])

print(classification_report(
    truths, predictions,
    target_names=["I", "N"]))

              precision    recall  f1-score   support

           I       0.98      0.98      0.98       832
           N       0.91      0.90      0.91       176

    accuracy                           0.97      1008
   macro avg       0.95      0.94      0.94      1008
weighted avg       0.97      0.97      0.97      1008



In [ ]:
print("F1_score of the model : {:0.2f}".format(f1_score(truths, predictions)))

F1_score of the model : 0.91


# Resource link-
*  [Albert Au Yeung](https://albertauyeung.github.io/2017/06/17/python-sequence-labelling-with-crf.html)
*  [An Introduction to Conditional Random Fields](http://homepages.inf.ed.ac.uk/csutton/publications/crftut-fntpdf)
* [icmla-conference on CRF](https://www.icmla-conference.org/icmla10/CFP_Tutorial_files/hakan.pdf)
*  [Introduction to Conditional Random Fields](http://blog.echen.me/2012/01/03/introduction-to-conditional-random-fields/)